In [ ]:
#결국 origin_price1를 구하는게 목표
#정가 알 수 있는거
data.loc[data.discount_type.isin(['골드회원+음반패키지','골드회원+음반 패키지','골드회원 할인+음반패키지']),'price']=data.loc[data.discount_type.isin(['골드회원+음반패키지','골드회원+음반 패키지','골드회원 할인+음반패키지']),'price']-10000
data.loc[data.discount_type.isin(['골드회원+음반패키지','골드회원+음반 패키지','골드회원 할인+음반패키지']),'discount_percent']=30
data.loc[data.discount_type.isin(['블랙프라이데이 이벤트']),'discount_percent']=40
data.loc[data.discount_type.isin(['싹틔우미 할인']),'discount_percent']=60
data.loc[data.discount_type.isin(['특판A']),'discount_percent']=20
data.loc[data.discount_type.isin(['특판B']),'discount_percent']=30
data.loc[data.discount_type.isin(['특판D']),'discount_percent']=50

origin_price1=[]      
for j in data.index.tolist():
    price = data.loc[j, 'price']
    calculated_price1 = price * (1/(1-(data.loc[j,'discount_percent'] / 100))) if not pd.isna(data.loc[j,'discount_percent']) else None
    origin_price1.append(calculated_price1)    
data['origin_price1']=origin_price1

data.loc[data.discount_type.isin(['초/중/고/대학생 할인(본인만)']),'origin_price1']=data.loc[data.discount_type.isin(['초/중/고/대학생 할인(본인만)']),'price']+5000

#정가는 알 수 없지만 가격이 고정된 경우(origin_price2에 넣어볼까 하고 일단 넣어봤음)
import numpy as np
origin_price2=[np.nan]*data.shape[0]
data.origin_price2=origin_price2
data.loc[data.discount_type.isin(['초/중/고등학생 할인(본인만)','기획사할인','초/중/고등학생 할인','하비에르 국제학교 학부모, 직원','하비에르 국제학교 재학생','기획사']),'origin_price2']=data.loc[data.discount_type.isin(['초/중/고등학생 할인(본인만)','기획사할인','초/중/고등학생 할인','하비에르 국제학교 학부모, 직원','하비에르 국제학교 재학생','기획사']),'price']

#정가는 알 수 없지만 좌석별 가격은 고정된 경우 ->할지말지 고민중

In [ ]:
#할인 역적용
import re
def percentage_numbers(text):
    if '%' in text:
        pattern = r'(\d+)\s*%'
        matches = re.findall(pattern, text)
        return int(matches[-1])
    else:
        if text=='일반':
            return 0
        pass

data['discount_percent']=data['discount_type'].apply(percentage_numbers)

#정가 측정 : %있는 것들 바꿔주기
origin_price1=[]
        
for j in data.index.tolist():
    price = data.loc[j, 'price']
    calculated_price1 = price * (1/(1-(data.loc[j,'discount_percent'] / 100))) if not pd.isna(data.loc[j,'discount_percent']) else None
    origin_price1.append(calculated_price1)    

data_d['origin_price1']=origin_price1


In [ ]:
#seat level(### 오류 나면 data_d.iloc[j,35:46]에 숫자 한번 바꿔봐)
data_d=data.copy()
import numpy as np
seat_level=[None]*data_d.shape[0]
for j in range(data_d.shape[0]):
    if (np.isnan(data_d.loc[j,'origin_price1']))&(set(data_d.iloc[j,35:46].tolist())=={0}):
        seat_level[j]='free'
    elif (np.isnan(data_d.loc[j,'origin_price1']))&(set(data_d.iloc[j,35:46].tolist())!={0}):
        seat_level[j]='unknown'
    else:   
        diff=[(i-int(data_d.loc[j,'origin_price1'])) for i in data_d.iloc[j,35:46].tolist() if not np.isnan(i)]
        if len([i for i in diff if ((i>=0)&(i<=2000))])==1:
            diff_value=[i for i in diff if ((i>=0)&(i<=2000))][0]
            seat_level[j]=data_d.columns[(data_d.iloc[j,35:46].tolist().index(int(data_d.loc[j,'origin_price1'])+diff_value)+35)]
        elif len([i for i in diff if ((i>=0)&(i<=2000))])==0:
            seat_level[j]='missing'
        else:
            seat_level[j]='more'

data_d['seat_level']=seat_level
data_d.loc[(data_d['seat_level']=='unknown')&(data_d['price']==0),'seat_level']='free'